In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [ ]:
import google.generativeai as genai

genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)

Artificial intelligence (AI) is a broad field encompassing many techniques, but at its core, it aims to create systems that can perform tasks that typically require human intelligence.  These tasks include learning, reasoning, problem-solving, perception, and natural language understanding.  Here's a breakdown of how various AI approaches work:

**1. Machine Learning (ML):** This is the most prevalent approach to AI today. Instead of explicitly programming rules, ML algorithms learn patterns from data.  There are several types:

* **Supervised Learning:** The algorithm learns from labeled data – data where the input and desired output are provided.  For example, showing an image classifier thousands of pictures of cats and dogs, labeled accordingly, allows it to learn to distinguish between them.  Examples include classification (categorizing data) and regression (predicting continuous values).

* **Unsupervised Learning:** The algorithm learns from unlabeled data, identifying patterns

# test sur transcription

In [ ]:
from bson import ObjectId


def get_transcription(collection, oid, verbose=False):
    # Récupération du document
    document_entry = collection.find_one({"_id": ObjectId(oid)})

    if document_entry is None:
        if verbose:
            print(f"Document avec l'oid {oid} non trouvé")
        return None

    return document_entry.get("whisper")

In [ ]:
from connection import get_collection

DB_HOST = "localhost"
collection = get_collection(DB_HOST)
oid = "6773e32258fc5717f3516b99"

transciption = get_transcription(collection, oid)

In [ ]:
prompt = f"""

Je vais te donner la transcription d'un episode d'une emission de radio qui s'appelle le masque et la plume sur France Inter.
Cet episode dure 1h et porte sur des livres. Il y a des intervenants qui parlent des livres qu'ils ont lus. Ils ne sont parfois pas d'accord.

Voici la transcription:
{transciption}

Je veux que tu identifies l'ensemble des livres dont on parle dans cette emission.
Et que tu me restitues cette liste de livres en separant auteur et titre. Si l'editeur est mentionne tu peux aussi le noter.

Concernant les avis des critiques, je veux que tu en fasses une forme de synthese en donnant une note de 1 à 10 (1 etant la note la plus basse et 10 la note la plus haute) utilisant les regles suivantes:
- la note 1 est vraiment pour les livres a eviter, les purges
- la note 10 est pour les livres a lire absolument, les chefs d'oeuvre
- la note 9 est pour les livres excellents, 
- la note 5 est pour les livres moyens, sans plus. pas horrible mais pas genial non plus
- les notations seront assez severes, ne garde la note 10 vraiment que pour les chefs d'oeuvre
- si un seul critique donne son avis, tu prendras sa note
- si plusieurs critiques se prononcent, tu prendras la moyenne de leurs notes
Je veux que tu conserves l'avis de chaque critique avec son prenom et son nom.
et que tu donnes la note moyenne obtenue pour chaque livre.
tu rajouteras une colonne pour dire le nombre de critiques qui ont donne leur avis sur le livre.
Puis si un des critiques a vraiment adore le livre (ce qui correspond a une note de 9 ou 10), tu mentionneras les noms des critiques dans une colonne "coup de coeur" a part.
Enfin si un des critiques fait etat d'un chef d'oeuvre (note 10), tu mentionneras cela dans une colonne "chef d'oeuvre" a part.
Tu me restitueras cette liste sous la forme d'un tableau au format markdown.

Et tu me donneras aussi le code python pour inserer ce tableau dans une base de donnees MongoDB "masque_et_la_plume" en separant les collections : auteurs, livres, editeurs, critiques, avis.
"""

In [ ]:
response = model.generate_content(prompt)
print(response.text)

Voici la liste des livres mentionnés dans l'émission, séparant auteur et titre, avec l'éditeur si mentionné :

* **Marc Dugain**, *L'avion, Poutine, l'Amérique et moi* (Albin Michel)
* **Emmanuel Lambert**, *Aucun respect* (Stock)
* **Emile Ferris**, *Moi, ce que j'aime, c'est les monstres* (tome 2) (Monsieur Toussaint L'Ouverture)
* **Fabrice Carreau**, *Fort Alamo* (Gallimard)
* **Mathieu Palin**, *Les Hommes manquent de courage* (L'Iconoclaste)
* **H.P. Lovecraft**, *Œuvres* (Gallimard, collection La Pléiade)
* **Xavier Lardoux**, *Christophe Honoré. Des fantômes et des arts*
* **Johan Zarka**, *Clean* (Les Éditions de la Couture)
* **Alain Weinstein**, *Chien perdu et autres chiens trouvés* (Flammarion)
* **Nick Hornby**, *Dickens et Prince. Un génie bien particulier* (Stock)
* **Sandrine Collette**, *Madeleine avant l'aube*


Voici le tableau récapitulatif des avis des critiques avec les notes et les colonnes supplémentaires demandées :

| Auteur                     | Titre       

| Auteur                     | Titre                                          | Patricia Martin | Jean-Marc Proust | Raphaël Léris | Laurent Chalumeau | Note Moyenne | Nb Critiques | Coup de Coeur | Chef d'œuvre |
|-----------------------------|-------------------------------------------------|-----------------|--------------------|-----------------|--------------------|---------------|-------------|----------------|--------------|
| Marc Dugain                 | L'avion, Poutine, l'Amérique et moi             | 9               | 5                  | 6               | 5                  | 6.25          | 4           | Patricia      |              |
| Emmanuel Lambert            | Aucun respect                                   |                  | 8                  | 9               | 9                  | 8.67          | 3           | Raphaël, Laurent |              |
| Emile Ferris                | Moi, ce que j'aime, c'est les monstres (tome 2) | 7               | 10                 | 9               | 6                  | 8           | 4           | Jean-Marc, Raphaël | Jean-Marc    |
| Fabrice Carreau             | Fort Alamo                                      | 9               | 7                  | 8               | 9                  | 8.25          | 4           | Patricia, Laurent |              |
| Mathieu Palin               | Les Hommes manquent de courage                 | 7               | 5                  | 5               | 7                  | 6           | 4           |              |              |
| H.P. Lovecraft             | Œuvres (Pléiade)                              |                  | 8                  | 7               |                  | 7.5           | 2           |              |              |
| Xavier Lardoux              | Christophe Honoré. Des fantômes et des arts     |                  |                  | 8               |                  | 8             | 1           | Raphaël      |              |
| Johan Zarka                 | Clean                                          |                  |                  |                  | 9                  | 9             | 1           | Laurent      |              |
| Alain Weinstein             | Chien perdu et autres chiens trouvés           | 9               |                  |                  |                  | 9             | 1           | Patricia      |              |
| Nick Hornby                 | Dickens et Prince. Un génie bien particulier   |                  |                  |                  | 9                  | 9             | 1           | Laurent      |              |
